# Build a Dashboard Application with Plotly Dash

In [1]:
#pip install pandas dash

In [2]:
import pandas as pd
from dash import Dash, dcc,html, Input, Output

import plotly.express as px

In [3]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

C:\Users\adm\AppData\Local\Temp\ipykernel_23352\1855361594.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\adm\AppData\Local\Temp\ipykernel_23352\1855361594.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [4]:
#pip install wget

In [5]:
import wget

In [6]:
url1="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
url2="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"
wget.download(url1, "spacex_launch_dash.csv")

100% [................................................................................] 2476 / 2476

'spacex_launch_dash.csv'

In [7]:
wget.download(url2, "spacex_dash_app.py")

100% [................................................................................] 2110 / 2110

'spacex_dash_app.py'

In [8]:
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [9]:
# Create a dash application
app = dash.Dash(__name__)

In [10]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id = 'site-dropdown',
                                            options = [
                                                         {'label': 'ALL SITES', 'value': 'ALL'},
                                                         {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                         {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                         {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                         {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                                      ],
                                            value = 'ALL',
                                            placeholder = "Select Launch Site", 
                                            searchable = True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id = 'payload-slider',
                                                min = 0, max = 10000, step = 1000,
                                                value = [min_payload, max_payload],
                                                marks = {0: '0 kg', 
                                                         2500: '2500 kg',
                                                         5000: '5000 kg', 
                                                         7500: '7500 kg', 
                                                         10000: '10000 kg'}
                                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [11]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id = 'success-pie-chart', component_property = 'figure'),
    Input(component_id = 'site-dropdown', component_property = 'value')
) 

def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        piechart = px.pie(data_frame = spacex_df,
                          names = 'Launch Site', 
                          values = 'class', 
                          title = 'Launches from All Sites'
                         )
        return piechart
    else:
        selected_data = spacex_df.loc[spacex_df['Launch Site'] == entered_site]
        piechart = px.pie(data_frame = selected_data, 
                          names = 'class', 
                          title = 'Launches from ' + entered_site
                         )
        return piechart

In [12]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id = 'success-payload-scatter-chart', component_property='figure'),
    [Input(component_id = 'site-dropdown', component_property = 'value'),
     Input(component_id = 'payload-slider', component_property = 'value')]
)

def get_scatter_plot(entered_site, payload):
    if entered_site == 'ALL':
        filtered_data = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload[0])
                                  & (spacex_df['Payload Mass (kg)'] <= payload[1])]
        scatterplot = px.scatter(data_frame = filtered_data, 
                                 x = "Payload Mass (kg)", 
                                 y = "class", 
                                 color = "Booster Version Category"
                                )
        return scatterplot
    else:
        selected_data = spacex_df.loc[spacex_df['Launch Site'] == entered_site]
        filtered_data = selected_data[(selected_data['Payload Mass (kg)'] >= payload[0])
                                      & (selected_data['Payload Mass (kg)'] <= payload[1])]
        scatterplot = px.scatter(data_frame = filtered_data, 
                                 x = "Payload Mass (kg)", 
                                 y = "class", 
                                 color = "Booster Version Category"
                                )
        return scatterplot

In [13]:
# Run the app
if __name__ == '__main__':
    app.run_server()